# CNNbasic Assignment#1


마크다운과 코드 속 `???` 를 채워주시면 됩니다!-!


# AlexNet

모델 구조 사진과 논문 사이트입니다.

- paper: http://www.cs.toronto.edu/~fritz/absps/imagenet.pdf

- dataset: http://image-net.org/challenges/LSVRC/2012/index#task

- Model architecture

![image.architecture1](attachment:image.png)

![model_architecture2](https://t1.daumcdn.net/cfile/tistory/99FEB93C5C80B5192E)

The second convolutional layer takes as input the (response-normalized
and pooled) output of the first convolutional layer and filters it with 256 kernels of size 5 × 5 × 48.
The third, fourth, and fifth convolutional layers are connected to one another without any intervening
pooling or normalization layers. The third convolutional layer has 384 kernels of size 3 × 3 ×
256 connected to the (normalized, pooled) outputs of the second convolutional layer. The fourth
convolutional layer has 384 kernels of size 3 × 3 × 192 , and the fifth convolutional layer has 256
kernels of size 3 × 3 × 192. The fully-connected layers have 4096 neurons each.


### Naive Version
CONV_1 - POOL_1 - CONV_2 - POOL_2 - CONV_3 - CONV_4 - CONV_5 - POOL_3 - FC1 - FC2 - FC3 (->SOFTMAX)

### detailed
CONV_1(ReLU) - POOL_1 - CONV_2(ReLU) - POOL_2 - CONV_3(ReLU) - CONV_4(ReLU) - CONV_5(ReLU) - POOL_3 -(Flatten) FC1(ReLU) - FC2(ReLU) - FC3(->SOFTMAX)

* ImageNet의 train, valid 데이터셋 얻는 방법  

출처: https://ndb796.tistory.com/471  

```
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 데이터 전처리 및 로딩
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageNet(root='path/to/imagenet', split='train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
```


$$Output_{size} = \frac{Input_{size} - Filter_{size} + 2\cdot Padding}{Stride} +1$$

## Layer 1 is a Convolution Layer_1

- **Input Image size**     224x224

- **Number of filters**     96

- **Filter size**     11x11

- **Stride**     4

- **Layer 1 Output**    round( (224 - 11 + 2 *0)/4 + 1) = 55

In [3]:
import numpy as np
import torch
import torch.nn as nn
from torchsummary import summary as summary_
import torch.nn.init as init

In [4]:
# Convolution + BatchNormalization + Relu 정의하기
def CBR2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True):
    layers = []
    layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                            kernel_size=kernel_size, stride=stride, padding=padding,
                            bias=bias)]
    layers += [nn.BatchNorm2d(num_features=out_channels)]
    layers += [nn.ReLU()]

    cbr = nn.Sequential(*layers)

    return cbr

In [12]:
def fc(input_size_fc, output_size_fc):
    layers = []
    layers += [nn.Linear(input_size_fc, output_size_fc)]
    layers += [nn.Dropout()]
    layers += [nn.ReLU()]

    fc = nn.Sequential(*layers)

    return fc

In [6]:
## TODO ##

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Conv_1 = CBR2d(in_channels=3, out_channels=96,
                                 kernel_size=11, stride=4, padding=0,
                                 bias=True)

In [7]:
Conv_1

Sequential(
  (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
  (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
)

$$ Output_{size} = round(\frac{Input_{size} - Pooling_{size}}{Stride} + 1 )$$


- Stride (스트라이드): 필터(커널)가 입력 데이터를 얼마나 건너뛰며 이동할지를 결정함. 예로, Stride가 1이면 한 칸씩 이동하고, Stride가 2이면 두 칸씩 이동을 의미. 예를 들어, 스트라이드가 2인 경우, 필터는 입력 데이터에서 2칸씩 이동하면서 연산을 수행함.  

- Kernel Size (커널 크기): 필터(커널)의 크기를 의미하며, 커널은 입력 데이터에 적용되어 특징을 추출하는 역할을 함. 커널 크기가 3x3이라면 3x3 크기의 작은 윈도우로 입력 데이터를 스캔하여 특징을 추출함을 의미함.  


## Layer 2 is a Max Pooling_1 Followed by Convolution_1

- **Input**     55

- **Max pooling**     1

- **Pooling size** (overlapping)      3

    * overlapping 중첩되어서 지나감

- **Stride**     2

- **Layer 2 Output**     round( (55-3) / 2 +1 ) = 27
    
    * 차원은 변함 없음

In [8]:
## TODO

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Max_pool_1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
Max_pool_1

MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)

## Layer 3 is a Convolution Layer_2

- **Input**     27

- **Number of filters**     96

- **Filter size**     5x5

- **Stride**     1

- **padding**     2

- **Layer 3 Output**    round( (27 - 5 + 2 *2)/1 + 1) =   27

In [ ]:
## TODO

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Conv_2 = CBR2d(in_channels=96, out_channels=256,
                                 kernel_size=5, stride=1, padding=2,
                                 bias=True)
Conv_2

## Layer 4 is a Max Pooling_2 Followed by Convolution_2

- **Input**     27

- **Max pooling**    1

- **Pooling size**(overlapping)     3

- **Stride**     2

- **Layer 4 Output**     round( (27-3) / 2 +1 ) =  13

In [ ]:
## TODO

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Max_pool_2 =  nn.MaxPool2d(kernel_size=3, stride=2)
Max_pool_2

## Layer 5 is a Convolution Layer_3

- **Input**     13

- **Number of filters**     384

- **Filter size**     3

- **Stride**     1

- **padding**    1

- **Layer 5 Output**  round((13-3 +2*1 )/1 +1) =  13


In [ ]:
## TODO

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Conv_3 = CBR2d(in_channels=256, out_channels=384,
                                 kernel_size=3, stride=1, padding=1,
                                 bias=True)
Conv_3

## Layer 6 is  a Convolution Layer_4

- **Input**     13

- **Number of filters**     384

- **Filter size**     3

- **Stride**     1

- **padding**     1

- **Layer 6 Output**  round((13-3+2*1)/1+1) = 13

In [ ]:
## TODO

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Conv_4 = CBR2d(in_channels=384, out_channels=384,
                                 kernel_size=3, stride=1, padding=1,
                                 bias=True)
Conv_4

## Layer 7 is a Convolution Layer_5

- **Input**     13

- **Number of filters**     256

- **Filter size**     3

- **Stride**     1

- **padding**     1

- **Layer 7 Output**  round((13-3+2*1)/1+1) = 13

In [ ]:
## TODO

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Conv_5 = CBR2d(in_channels=384, out_channels=256,
                                 kernel_size=3, stride=1, padding=1,
                                 bias=True)
Conv_5

## Layer 8 is a Max Pooling_3 Followed by Convolution_5

- **Input**     13

- **Max pooling**     1  

- **Pooling size**(overlapping)     3

- **Stride**     2

- **Layer 8 Output**   round( (13-3)/2 +1) =  6

In [ ]:
## TODO

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Max_pool_3 = nn.MaxPool2d(kernel_size=3, stride=2)
Max_pool_3

## Layer 9 is a Fully_Connected layer_1

- **input**     6x6x256

- **flatten**     9216

- **output size**     4096

- **N** Number of input data     9216

In [13]:
## TODO

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

FC1 = fc(6*6*256, 4096)
FC1

Sequential(
  (0): Linear(in_features=9216, out_features=4096, bias=True)
  (1): Dropout(p=0.5, inplace=False)
  (2): ReLU()
)

## Layer 10 is a Fully_Connected layer_2

- **input**     4096

- **output size**     4096

- **N** Number of input data =      4096

In [ ]:
## TODO

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

FC2 = fc(4096,4096)
FC2

## Layer 11 is a Fully_Connected layer_3

- **input**     4096

- **output size**     1000

- **N** Number of input data =      4096

- **Num_classes** Number of labels =      1000

In [ ]:
## TODO

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

FC3 = fc(4096,1000)

FC3

In [ ]:
import torch.nn.functional as F

# 모델 출력에 softmax를 적용한 후 손실을 계산하는 방법
'''loss_with_softmax = nn.CrossEntropyLoss()(F.softmax(FC3, dim=1), labels) #labels: validation of image'''